In [8]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv("final_performance_v4.csv", parse_dates=['방송일시','date', 'real_date'])
data

,Unnamed: 0,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,date,...,g_3,s_0,s_1,w_0,w_1,w_2,w_3,w_4,w_5,cpi
0,0,2019-01-01 06:00:00,20.0,296,891,테이트 남성 셀린니트3종,6,39900,2099000.0,2019-01-01,...,0,0,0,0,0,0,0,0,0,104.03
1,1,2019-01-01 06:00:00,20.0,296,898,테이트 여성 셀린니트3종,6,39900,4371000.0,2019-01-01,...,0,0,0,0,0,0,0,0,0,104.03
2,2,2019-01-01 06:20:00,20.0,296,891,테이트 남성 셀린니트3종,6,39900,3262000.0,2019-01-01,...,0,1,0,0,0,0,0,0,0,104.03
3,3,2019-01-01 06:20:00,20.0,296,898,테이트 여성 셀린니트3종,6,39900,6955000.0,2019-01-01,...,0,1,0,0,0,0,0,0,0,104.03
4,4,2019-01-01 06:40:00,20.0,296,891,테이트 남성 셀린니트3종,6,39900,6672000.0,2019-01-01,...,0,0,1,0,0,0,0,0,0,104.03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35374,35374,2019-12-31 23:40:00,20.0,374,1130,일시불쿠첸압력밥솥 6인용,9,148000,10157000.0,2019-12-31,...,1,1,0,0,0,0,0,0,0,105.51
35375,35375,2020-01-01 00:00:00,20.0,374,1124,무이자쿠첸압력밥솥 10인용,9,178000,50929000.0,2019-12-31,...,1,0,1,0,0,0,0,0,0,106.26
35376,35376,2020-01-01 00:00:00,20.0,374,1129,일시불쿠첸압력밥솥 10인용,9,168000,104392000.0,2019-12-31,...,1,0,1,0,0,0,0,0,0,106.26
35377,35377,2020-01-01 00:00:00,20.0,374,1125,무이자쿠첸압력밥솥 6인용,9,158000,13765000.0,2019-12-31,...,1,0,1,0,0,0,0,0,0,106.26


In [3]:
import sklearn
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split
from tqdm.auto import tqdm
import gc

In [4]:
use_var = ['마더코드', '상품코드', '상품군', 'week', 'hour', 'weekofyear', 'stage', 'prime', 'IsHoliday', '노출(분)', '판매단가', '취급액', '지속휴일수', 'TEMP', 'HUM', 'cpi']
Data = data[use_var]

In [5]:
X = Data.drop(['취급액'], axis = 1)
Y = Data[['취급액']]
print("data shape : ", X.shape)
print("label shape : " , Y.shape)

data shape :  (35379, 15)
label shape :  (35379, 1)


In [6]:
x_train, x_valid, y_train, y_valid = train_test_split(X, Y, test_size = 0.1, random_state = 123457)
print("x_train shape : ", x_train.shape)
print("y_train shape : ", y_train.shape)
print("x_valid shape : ", x_valid.shape)
print("y_valid shape : ", y_valid.shape)

x_train shape :  (31841, 15)
y_train shape :  (31841, 1)
x_valid shape :  (3538, 15)
y_valid shape :  (3538, 1)


## Make train data

In [10]:
make_train_cols = ['상품코드', '상품군', 'week', 'hour', 'weekofyear', 'stage', 'IsHoliday', '노출(분)', '판매단가', '지속휴일수', 'TEMP', 'HUM', 'cpi']
to_cat = ['상품코드', '상품군', 'week', 'hour', 'weekofyear', 'stage']
X_train_input = np.array(x_train['prime']).reshape(-1,1)
for col in make_train_cols:
    if col in to_cat:
        x_train[col] = x_train[col].astype('category') 
        inputs = np.array(x_train[col]).reshape(-1,1)
        X_train_input = np.concatenate((X_train_input,inputs), axis=1)
        print(col, inputs[0].shape, X_train_input[0].shape)
    else:
        inputs = np.array(x_train[col]).reshape(-1,1)
        X_train_input = np.concatenate((X_train_input,inputs), axis=1)
        print(col, inputs[0].shape, X_train_input[0].shape)
print("X_train_input shape : ", X_train_input.shape)

C:\ProgramData\Anaconda3\envs\tensor\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


상품코드 (1,) (2,)
상품군 (1,) (3,)
week (1,) (4,)
hour (1,) (5,)
weekofyear (1,) (6,)
stage (1,) (7,)
IsHoliday (1,) (8,)
노출(분) (1,) (9,)
판매단가 (1,) (10,)
지속휴일수 (1,) (11,)
TEMP (1,) (12,)
HUM (1,) (13,)
cpi (1,) (14,)
X_train_input shape :  (31841, 14)


## Make valid data

In [ ]:
pd.set_option('mode.chained_assignment',  None)
def goods_code_encoding(value, encoder, price, mother, cat):
    if np.any(encoder.categories_[0] == value):
        return encoder.transform([[value]]).toarray()
    else:
        if mother in x_train['마더코드']:
            frame = x_train[x_train['마더코드']==mother]
            frame['diff'] =  abs(frame['판매단가'] - price)
            goods_code = frame[frame['diff']==min(frame['diff'])].iloc[0,1]
            return encoder.transform([[goods_code]]).toarray()
        else:
            frame = x_train[x_train['상품군']==cat]
            frame['diff'] =  abs(frame['판매단가'] - price)
            goods_code = frame[frame['diff']==min(frame['diff'])].iloc[0,1]
            return encoder.transform([[goods_code]]).toarray()        

In [ ]:
make_valid_cols = ['상품코드', '상품군', 'week', 'hour', 'weekofyear', 'stage', 'IsHoliday', '노출(분)', '판매단가', '지속휴일수', 'TEMP', 'HUM', 'cpi']
to_cat = ['상품코드', '상품군', 'week', 'hour', 'weekofyear', 'stage']
X_valid_input = np.array(x_valid['prime']).reshape(-1,1)
for col in make_valid_cols:
    if col == '상품코드':
        row = x_valid.iloc[0]
        inputs = goods_code_encoding(row['상품코드'], encoders['상품코드'], row['판매단가'], row['마더코드'], row['상품군'])
        for i in range(1, len(x_valid)):
            row = x_valid.iloc[i]
            vector = goods_code_encoding(row['상품코드'], encoders['상품코드'], row['판매단가'], row['마더코드'], row['상품군'])
            inputs = np.append(inputs,vector, axis=0)
        X_valid_input = np.concatenate((X_valid_input,inputs), axis=1)
        print(col, inputs[0].shape, X_valid_input[0].shape)
    elif col in to_cat:
        onehot = encoders[col].transform(np.array(x_valid[col]).reshape(-1,1)).toarray()
        X_valid_input = np.concatenate((X_valid_input,onehot), axis=1)
        print(col, onehot[0].shape, X_valid_input[0].shape)
    else:
        inputs = np.array(x_valid[col]).reshape(-1,1)
        X_valid_input = np.concatenate((X_valid_input,inputs), axis=1)
        print(col, onehot[0].shape, X_valid_input[0].shape)
print("X_valid_input shape : ", X_valid_input.shape)